In [1]:
import numpy as np
import cv2
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline


WAVELET TRANFORMATION

In [2]:
import numpy as np
import pywt
import cv2    

def w2d(img, mode='haar', level=1):
    imArray = img
    #Datatype conversions
    #convert to grayscale
    imArray = cv2.cvtColor( imArray,cv2.COLOR_RGB2GRAY )
    #convert to float
    imArray =  np.float32(imArray)   
    imArray /= 255;
    # compute coefficients 
    coeffs=pywt.wavedec2(imArray, mode, level=level)

    #Process Coefficients
    coeffs_H=list(coeffs)  
    coeffs_H[0] *= 0;  

    # reconstruction
    imArray_H=pywt.waverec2(coeffs_H, mode);
    imArray_H *= 255;
    imArray_H =  np.uint8(imArray_H)

    return imArray_H

In [4]:
def get_cropped(image_path):
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x,y,w,h) in faces:
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = img[y:y+h, x:x+w]
        eyes = eye_cascade.detectMultiScale(roi_gray)
        if len(eyes) >= 2:
            return roi_color

In [3]:
face_cascade = cv2.CascadeClassifier('./opencv/haarcascades/haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('./opencv/haarcascades/haarcascade_eye.xml')


In [5]:
path_to_data = "./dataset/"
path_to_cr_data = "./dataset/cropped/"
import os
img_dirs = []
for entry in os.scandir(path_to_data):
    if entry.is_dir():
        img_dirs.append(entry.path)

In [6]:
img_dirs

['./dataset/chritiano_ronaldo',
 './dataset/gukesh',
 './dataset/neeraj_chopra',
 './dataset/pv_sindhu',
 './dataset/shubhman_gill']

In [7]:
import shutil
if os.path.exists(path_to_cr_data):
     shutil.rmtree(path_to_cr_data)
os.mkdir(path_to_cr_data)

In [8]:
cropped_img_dirs = []
person_file_dict = {}
for img_dir in img_dirs:
    count = 1
    celebrity_name = img_dir.split('/')[-1]
    person_file_dict[celebrity_name] = []
    for entry in os.scandir(img_dir):
        roi_color = get_cropped(entry.path)
        if roi_color is not None:
            cropped_folder = path_to_cr_data + celebrity_name
            if not os.path.exists(cropped_folder):
                os.makedirs(cropped_folder)
                cropped_img_dirs.append(cropped_folder)
                print("Generating cropped images in folder: ",cropped_folder)
            cropped_file_name = celebrity_name + str(count) + ".png"
            cropped_file_path = cropped_folder + "/" + cropped_file_name
            cv2.imwrite(cropped_file_path, roi_color)
            person_file_dict[celebrity_name].append(cropped_file_path)
            count += 1

Generating cropped images in folder:  ./dataset/cropped/chritiano_ronaldo
Generating cropped images in folder:  ./dataset/cropped/gukesh
Generating cropped images in folder:  ./dataset/cropped/neeraj_chopra
Generating cropped images in folder:  ./dataset/cropped/pv_sindhu
Generating cropped images in folder:  ./dataset/cropped/shubhman_gill


error: OpenCV(4.12.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:199: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [9]:
person_file_dict = {}
for img_dir in cropped_img_dirs:
    person_name = img_dir.split('/')[-1]
    file_list = []
    for entry in os.scandir(img_dir):
        file_list.append(entry.path)
    person_file_dict[person_name] = file_list
person_file_dict

{'chritiano_ronaldo': ['./dataset/cropped/chritiano_ronaldo\\chritiano_ronaldo1.png',
  './dataset/cropped/chritiano_ronaldo\\chritiano_ronaldo10.png',
  './dataset/cropped/chritiano_ronaldo\\chritiano_ronaldo11.png',
  './dataset/cropped/chritiano_ronaldo\\chritiano_ronaldo12.png',
  './dataset/cropped/chritiano_ronaldo\\chritiano_ronaldo13.png',
  './dataset/cropped/chritiano_ronaldo\\chritiano_ronaldo14.png',
  './dataset/cropped/chritiano_ronaldo\\chritiano_ronaldo15.png',
  './dataset/cropped/chritiano_ronaldo\\chritiano_ronaldo16.png',
  './dataset/cropped/chritiano_ronaldo\\chritiano_ronaldo17.png',
  './dataset/cropped/chritiano_ronaldo\\chritiano_ronaldo18.png',
  './dataset/cropped/chritiano_ronaldo\\chritiano_ronaldo19.png',
  './dataset/cropped/chritiano_ronaldo\\chritiano_ronaldo2.png',
  './dataset/cropped/chritiano_ronaldo\\chritiano_ronaldo20.png',
  './dataset/cropped/chritiano_ronaldo\\chritiano_ronaldo21.png',
  './dataset/cropped/chritiano_ronaldo\\chritiano_ronaldo

In [10]:
class_dict = {}
count = 0
for person_name in person_file_dict.keys():
    class_dict[person_name] = count
    count = count + 1
class_dict

{'chritiano_ronaldo': 0,
 'gukesh': 1,
 'neeraj_chopra': 2,
 'pv_sindhu': 3,
 'shubhman_gill': 4}

In [11]:
X, y = [], []
for person_name, training_files in person_file_dict.items():
    for training_image in training_files:
        img = cv2.imread(training_image)
        scalled_raw_img = cv2.resize(img, (32, 32))
        img_har = w2d(img,'db1',5)
        scalled_img_har = cv2.resize(img_har, (32, 32))
        combined_img = np.vstack((scalled_raw_img.reshape(32*32*3,1),scalled_img_har.reshape(32*32,1)))
        X.append(combined_img)
        y.append(class_dict[person_name])  

MODEL TRAINING : SVM AND LOGISTIC REGRESSION

In [12]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

In [13]:
from sklearn.metrics import classification_report

In [14]:
X = np.array(X).reshape(len(X),4096).astype(float)
X.shape

(103, 4096)

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

pipe = Pipeline([('scaler', StandardScaler()), ('svc', SVC(kernel = 'rbf', C = 10))])
pipe.fit(X_train, y_train)
pipe.score(X_test, y_test)

0.7692307692307693

In [16]:
from sklearn import svm

from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV

In [17]:
model_params = {
    'svm': {
        'model': svm.SVC(gamma='auto',probability=True),
        'params' : {
            'svc__C': [1,10,100,1000],
            'svc__kernel': ['rbf','linear']
        }  
    },
    'logistic_regression' : {
        'model': LogisticRegression(solver='liblinear',multi_class='auto'),
        'params': {
            'logisticregression__C': [1,5,10]
        }
    }
}

In [22]:
scores = []
best_estimators = {}
import pandas as pd
for algo, mp in model_params.items():
    pipe = make_pipeline(StandardScaler(), mp['model'])
    clf =  GridSearchCV(pipe, mp['params'], cv=5, return_train_score=False)
    clf.fit(X_train, y_train)
    scores.append({
        'model': algo,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })
    best_estimators[algo] = clf.best_estimator_
    
df = pd.DataFrame(scores,columns=['model','best_score','best_params'])
df

c:\MINOR_PROJECT\Classifier\venv\Lib\site-packages\sklearn\model_selection\_split.py:811: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(
c:\MINOR_PROJECT\Classifier\venv\Lib\site-packages\sklearn\model_selection\_split.py:811: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(
c:\MINOR_PROJECT\Classifier\venv\Lib\site-packages\sklearn\linear_model\_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
c:\MINOR_PROJECT\Classifier\venv\Lib\site-packages\sklearn\linear_model\_logistic.py:1296: FutureWarning: Using the 'liblinear' solver for multiclass classification is deprecated. An error will be raised in 1.8. Either use another solver which supports the multinomial loss or wrap the estimator in a

,model,best_score,best_params
0,svm,0.767500,"{'svc__C': 1, 'svc__kernel': 'linear'}"
1,logistic_regression,0.870833,{'logisticregression__C': 1}


In [23]:
best_estimators

{'svm': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('svc',
                  SVC(C=1, gamma='auto', kernel='linear', probability=True))]),
 'logistic_regression': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('logisticregression',
                  LogisticRegression(C=1, multi_class='auto',
                                     solver='liblinear'))])}

In [24]:
best_estimators['svm'].score(X_test,y_test)

0.7692307692307693

In [25]:
best_estimators['logistic_regression'].score(X_test,y_test)

0.6153846153846154

In [26]:
best_clf = best_estimators['svm']

In [27]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, best_clf.predict(X_test))
cm

array([[6, 0, 1, 1],
       [0, 5, 0, 0],
       [0, 0, 5, 4],
       [0, 0, 0, 4]])

In [28]:
import joblib 
# Save the model  
joblib.dump(best_clf, 'saved_model.pkl') 

['saved_model.pkl']

In [29]:
import json
with open("class_dictionary.json","w") as f:
    f.write(json.dumps(class_dict))